In [ ]:
# Install required libraries
!pip install -q --upgrade tensorflow keras keras-nlp transformers torch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
!pip install --upgrade kaggle kagglehub -q

In [ ]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
gemma_lm = GemmaCausalLM.from_preset("gemma_2b_en")

In [ ]:
# Define model name
MODEL_NAME = "gemma_2b_en"

# Initialize GemmaCausalLM with the preset
gemma_lm = GemmaCausalLM.from_preset(MODEL_NAME)

# Enable LoRA with rank=64
gemma_lm.backbone.enable_lora(rank=64)

# Set float precision to bfloat16
keras.backend.set_floatx("bfloat16")

print(f"Model '{MODEL_NAME}' initialized with LoRA and bfloat16 precision.")

Model 'gemma_2b_en' initialized with LoRA and bfloat16 precision.


In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,527,995,904 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,527,995,904 (9.42 GB)

 Trainable params: 21,823,488 (83.25 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [ ]:
DATASET_URL = "https://huggingface.co/datasets/vetHealthGuy/Veteran_Affairs_NorthEast_Region_Conversational_Dataset/resolve/main/Updated_VA_Facilities_Conversational_with_Phone.csv"

!wget -nv -nc -O vet_chat.csv $DATASET_URL

In [ ]:
# Dataset Path
NEW_DATASET_PATH = "vet_chat.csv"
PROMPT_TEMPLATE = "{Prompt} {Response}"

# Generate training data from CSV
def generate_training_data(training_ratio: int = 100) -> list[str]:
    """
    Generate formatted training data from a CSV file.

    Args:
    - training_ratio (int): Percentage of the dataset to use for training (1-100).

    Returns:
    - list[str]: List of formatted training examples.
    """
    assert 0 < training_ratio <= 100
    data = []
    with open(NEW_DATASET_PATH, newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row.get("context"):
                continue
            data.append(PROMPT_TEMPLATE.format(Prompt=row['Prompt'], Response=row['Response']))
    total_data_count = len(data)
    training_data_count = total_data_count * training_ratio // 100
    print(f"Training examples: {training_data_count}/{total_data_count}")
    return data[:training_data_count]

# Generate training data
training_data = generate_training_data(training_ratio=10)


Training examples: 329/3290


In [ ]:
def finetune_gemma(model: GemmaCausalLM, data: list[str]):
    """
    Fine-tune the GemmaCausalLM model.

    Args:
    - model: GemmaCausalLM model to fine-tune.
    - data: List of training examples as strings.
    """
    # Reduce the input sequence length to limit memory usage
    model.preprocessor.sequence_length = 64

    # Define early stopping
    early_stopping = EarlyStopping(
        monitor='loss',
        patience=2,  # Allow 2 epochs of no improvement before stopping
        restore_best_weights=True  # Restore the best model weights
    )

    # Configure the AdamW optimizer
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-6,
        weight_decay=0.01,
    )

    # Convert data to a tf.data.Dataset and batch it
    dataset = tf.data.Dataset.from_tensor_slices(data).batch(4)

    # Compile the model with greedy decoding for text generation
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
        sampler="greedy"  # Greedy decoding for text generation
    )

    # Train the model
    model.fit(dataset, epochs=4, callbacks=[early_stopping])

print("Fine-tuning function ready.")


Fine-tuning function ready.


In [ ]:
# Fine-tune the model using the finetune_gemma function
finetune_gemma(gemma_lm, training_data)

print("Fine-tuning complete.")


Epoch 1/4
83/83 ━━━━━━━━━━━━━━━━━━━━ 1046s 12s/step - loss: 17.7128 - sparse_categorical_accuracy: 0.5462 - weighted_sparse_categorical_accuracy: 0.5719
Epoch 2/4
83/83 ━━━━━━━━━━━━━━━━━━━━ 981s 12s/step - loss: 16.1674 - sparse_categorical_accuracy: 0.5331 - weighted_sparse_categorical_accuracy: 0.5582
Epoch 3/4
83/83 ━━━━━━━━━━━━━━━━━━━━ 972s 12s/step - loss: 13.2783 - sparse_categorical_accuracy: 0.5259 - weighted_sparse_categorical_accuracy: 0.5505
Epoch 4/4
83/83 ━━━━━━━━━━━━━━━━━━━━ 1031s 12s/step - loss: 9.8348 - sparse_categorical_accuracy: 0.4350 - weighted_sparse_categorical_accuracy: 0.4555
Fine-tuning complete.


In [ ]:
# Test prompts
TEST_PROMPTS = [
    "Where are services near Philadelphia?",
    "How can I apply for healthcare benefits as a veteran in Reading, Pa?",
    "Are there VA mental health resources at the Virginia",
    "Where is the nearest VA hospital in New York Queens borough?",
    "What is the phone number for the VA in Boston?"
]

In [ ]:
# Test the fine-tuned model
for prompt in TEST_PROMPTS:
    # Generate a response
    response = gemma_lm.generate(prompt, max_length=40)  # Adjust max_length as needed
    print(f"Prompt: {prompt}")
    print(f"Response: {response}\n{'-' * 40}")

Prompt: Where are services near Philadelphia?
Response: Where are services near Philadelphia?

The following is a list of services that are available to you.

* <strong>Online</strong>: You can access your account and make payments online.
*
----------------------------------------
Prompt: How can I apply for healthcare benefits as a veteran in Reading, Pa?
Response: How can I apply for healthcare benefits as a veteran in Reading, Pa?

Answer:

Step 1/5
1. First, you need to determine if you are eligible for healthcare
----------------------------------------
Prompt: Are there VA mental health resources at the Virginia
Response: Are there VA mental health resources at the Virginia Tech Carilion School of Medicine?

Yes, the VA has a number of resources for medical students and residents.

* The VA has a number
----------------------------------------
Prompt: Where is the nearest VA hospital in New York Queens borough?
Response: Where is the nearest VA hospital in New York Queens boroug

# Conclusion on Model Responses
The fine-tuned GemmaCausalLM model demonstrates some promising capabilities but falls short in providing accurate and contextually appropriate responses to the prompts. The following observations were made:

## Repetition of the Prompt:

The model frequently echoes the prompt in its response rather than generating meaningful content.
Example: For the prompt "Where are services near Philadelphia?", the response began by repeating the question verbatim.
Incomplete or Generic Responses:

The model provides incomplete answers that lack specific details or actionable information.
Example: For "How can I apply for healthcare benefits as a veteran in Reading, Pa?", the response starts with "Step 1/5" but does not proceed to provide further steps.
## Irrelevant Content:

Some responses include irrelevant or unrelated content, such as references to educational contexts or generic lists.
Example: For "What is the phone number for the VA in Boston?", the response unexpectedly shifted to mentioning the 2019-2020 school year and community college.
## Missed Opportunities for Specificity:

The model fails to leverage the fine-tuning data to provide specific answers, such as accurate phone numbers or locations.
Future Recommendations
While the current performance of the model is not fully aligned with expectations, it establishes a solid foundation for future development.
## To improve the model:

### Enhance the Training Dataset:
Include more diverse and detailed examples with well-defined prompts and accurate, complete responses.
### Increase Fine-Tuning Epochs:
Allow the model more time to learn from the fine-tuning dataset by increasing the number of epochs while monitoring for overfitting.
### Adjust Hyperparameters:
Experiment with learning rates, sequence lengths, and batch sizes to optimize performance.
### Incorporate Domain-Specific Tokenizers:
If needed, include a domain-specific tokenizer to better handle specialized language and context.
## Conclusion
While the model did not meet the requirements for this project, it showcases potential for generating structured responses and provides a good starting point for future enhancements. With further refinement and targeted adjustments, the model could become a valuable tool for answering veteran-focused queries effectively.

In [ ]:
!pip install google-cloud-storage -q

In [ ]:
!gcloud config get core/account

awdatascience@gmail.com


In [ ]:
# Authenticate the Cloud SDK with your credentials
# !gcloud auth login

# Authenticate code and libraries with your credentials
# !gcloud auth application-default login

In [ ]:
res = !gcloud config get core/project
PROJECT_ID = res[0]

print(f"{PROJECT_ID=}")

PROJECT_ID='velvety-study-440800-e0'


In [ ]:
# List your projects
# !gcloud projects list

# Define the default project
# PROJECT_ID = ""  # @param {type:"string"}
# !gcloud config set core/project $PROJECT_ID

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

!gcloud config set ai/region $REGION

Updated property [ai/region].


In [ ]:
# Define a bucket related to your project
BUCKET_URI = f"gs://gemma-{PROJECT_ID}-unique"
# Or use an existing one
# BUCKET_URI = "gs://"  # @param {type:"string"}

res = !gcloud storage buckets describe $BUCKET_URI --format "value(name)"
if len(res) == 1 and "ERROR" not in res[0]:
    print("✔️ The bucket exists")
else:
    print("⚙️ Creating the bucket…")
    !gcloud storage buckets create $BUCKET_URI --project $PROJECT_ID --location $REGION

⚙️ Creating the bucket…
Creating gs://gemma-velvety-study-440800-e0-unique/...


In [ ]:
BUCKET_URI

'gs://gemma-velvety-study-440800-e0-unique'

In [ ]:
BUCKET_PATH = f"{BUCKET_URI}/models"

In [ ]:
# Save the Keras model locally
gemma_lm.save("fine_tuned_gemma_model.keras")

In [ ]:
!gcloud storage rsync --recursive --verbosity error $HUGGINGFACE_MODEL_DIR $DEPLOYED_MODEL_URI

In [ ]:
# Save the model in H5 format locally
gemma_lm.save("fine_tuned_gemma_model.h5")

In [ ]:
from transformers import GemmaConfig, AutoModelForCausalLM

# Define Hugging Face Model Configuration
hf_config = GemmaConfig(
    vocab_size=30522,             # Match the vocabulary size of your model
    n_layer=12,                   # Number of transformer layers
    n_head=12,                    # Number of attention heads
    hidden_size=768,              # Hidden dimension size
    intermediate_size=3072,       # Feedforward layer dimension
    max_position_embeddings=512   # Max sequence length
)

# Directory to save the Hugging Face model
hf_model_dir = "huggingface_model"
os.makedirs(hf_model_dir, exist_ok=True)

# Save Hugging Face model
torch_model = AutoModelForCausalLM.from_config(hf_config)
torch_model.save_pretrained(hf_model_dir)

print(f"Model saved in Hugging Face format in directory '{hf_model_dir}'.")


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Model saved in Hugging Face format in directory 'huggingface_model'.


In [ ]:
MODEL_DIRS=["/content/keras_model", "/content/H5_Weights"]
DEPLOYED_MODEL_URI="gs://gemma-velvety-study-440800-e0-unique"

for model_dir in MODEL_DIRS:
    !gcloud storage rsync --recursive --verbosity error $model_dir $DEPLOYED_MODEL_URI


At file:///content/keras_model/**, worker process 71930 thread 131927862427648 listed 1...
At gs://gemma-velvety-study-440800-e0-unique/**, worker process 71930 thread 131927862427648 listed 6...
Copying file:///content/keras_model/fine_tuned_gemma_model.keras to gs://gemma-velvety-study-440800-e0-unique/fine_tuned_gemma_model.keras

Average throughput: 26.7MiB/s
At file:///content/H5_Weights/**, worker process 73229 thread 133674141175808 listed 1...
At gs://gemma-velvety-study-440800-e0-unique/**, worker process 73229 thread 133674141175808 listed 7...
Copying file:///content/H5_Weights/fine_tuned_gemma_model.h5 to gs://gemma-velvety-study-440800-e0-unique/fine_tuned_gemma_model.h5

Average throughput: 40.5MiB/s


### Loading the models.


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import AdamW

# Load the model from the .keras file
model = load_model("fine_tuned_gemma_model.keras", compile=False)

# Recompile the model (optional, depending on your use case)
model.compile(
    loss="sparse_categorical_crossentropy",  # Match the loss used during training
    optimizer=AdamW(learning_rate=5e-6),
    metrics=["accuracy"]
)

print("Model loaded and recompiled successfully.")


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 146 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded and recompiled successfully.


In [ ]:
model.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.models import load_model
from keras_nlp.models import GemmaCausalLM

# Load the model with the custom class
gemma_lm.load_weights("fine_tuned_gemma_model.h5")

print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,527,995,904 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,571,642,882 (9.58 GB)

 Trainable params: 21,823,488 (83.25 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

 Optimizer params: 43,646,978 (166.50 MB)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers import AutoModelForCausalLM

# Load the Hugging Face model  ##No tokenizer was used
model_dir = "/content/huggingface_model"
model = AutoModelForCausalLM.from_pretrained(model_dir)

print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
import torch

# Example input prompt (you need to convert text to token IDs based on your model's vocabulary)
prompt = "What services are available for veterans at the VA in Philadelphia?"

# Dummy input IDs (replace with actual token IDs if available)
# For example, [101] could represent a start token, and [102] an end token
input_ids = torch.tensor([[101, 2023, 2003, 1037, 5204, 102]])

# Generate output
outputs = model.generate(input_ids=input_ids, max_length=50)

print("Generated Response (token IDs):", outputs)


Generated Response (token IDs): tensor([[  101,  2023,  2003,  1037,  5204,   102, 24132, 24132, 24132, 24132,
         24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132,
         24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132,
         24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132,
         24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132, 24132]])


In [ ]:
# Example vocabulary with placeholder for unknown tokens
vocab = {
    101: "[START]",
    2023: "What",
    2003: "services",
    1037: "are",
    5204: "available",
    102: "[END]",
}

# Decode the output token IDs
decoded_response = " ".join([vocab.get(token.item(), "[UNK]") for token in outputs[0]])
print("Generated Response (decoded):", decoded_response)


Generated Response (decoded): [START] What services are available [END] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]


### What Happened

I didn' save a tokenizer or vocab.text  Since I do not have that much compute space or funding we will need to create something out of the dataset.

In [ ]:
!pip install sentencepiece -q

In [ ]:
import pandas as pd
import tempfile
import sentencepiece as spm

# Load the dataset
dataset_path = "vet_chat.csv"
data = pd.read_csv(dataset_path)

# Filter out rows with empty prompts or responses
data = data.dropna(subset=["Prompt", "Response"])

# Combine 'Prompt' and 'Response' as separate sentences, joined by newline
combined_text = "\n".join(data['Prompt'].astype(str) + " " + data['Response'].astype(str))
print(f"Combined text length: {len(combined_text)}")

# Write combined_text to a temporary file with proper sentence separation
with tempfile.NamedTemporaryFile(delete=False, mode="w", suffix=".txt") as temp_file:
    temp_file.write(combined_text)
    temp_file_path = temp_file.name

# Validate the temporary file contents
with open(temp_file_path, "r") as f:
    file_contents = f.read()
    print(f"Sample file contents:\n{file_contents[:500]}")  # Display first 500 characters for verification

# Train the SentencePiece model using a smaller vocab_size (<= 2549)
spm.SentencePieceTrainer.train(input=temp_file_path, model_prefix="spm_model", vocab_size=2500)

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor(model_file="spm_model.model")

# Display the vocabulary size
vocab_size = sp.get_piece_size()
print(f"Vocabulary size: {vocab_size}")


Combined text length: 651285
Sample file contents:
Where is the VA facility in Danbury, CT? The VA facility in Danbury, CT is located at 7 Germantown Road, Ste 2B   Danbury, CT 06810. You can contact them at 203-798-8422  .
Where is the VA facility in Danbury, CT? The VA facility in Danbury, CT is located at 131 West Street, Suite 2A, Second Floor   Danbury, CT 06810. You can contact them at 203-790-4000  .
Where is the VA facility in Rocky Hill, CT? The VA facility in Rocky Hill, CT is located at 25 Elm Street, Suite A   Rocky Hill, CT 06067. Y
Vocabulary size: 2500


In [ ]:
# Test encoding with the trained tokenizer
sample_prompt = "Where is the VA facility in Danbury CT?"
encoded = sp.encode(sample_prompt, out_type=int)  # Get token IDs
decoded = sp.decode(encoded)  # Convert token IDs back to text

print("Encoded token IDs:", encoded)
print("Decoded text:", decoded)


Encoded token IDs: [26, 10, 43, 3, 17, 5, 8, 14, 11, 12, 13, 6, 456, 118, 19]
Decoded text: Where is the VA facility in Danbury CT?


In [ ]:
print("Vocabulary size:", sp.get_piece_size())

Vocabulary size: 2500


In [ ]:
# Generate outputs
outputs = model.generate(input_tensor, max_length=50)

# Validate token IDs against SentencePiece vocabulary size
vocab_size = sp.get_piece_size()
output_ids = outputs[0].tolist()

if any(token_id >= vocab_size or token_id < 0 for token_id in output_ids):
    print("Generated token IDs exceed the tokenizer's vocabulary size.")
else:
    print("Generated token IDs are within the tokenizer's vocabulary size.")


Generated token IDs exceed the tokenizer's vocabulary size.


In [ ]:
def safe_decode_with_sentencepiece(ids, sp):
    # Replace out-of-range IDs with the [UNK] token
    vocab_size = sp.get_piece_size()
    safe_ids = [token_id if 0 <= token_id < vocab_size else sp.unk_id() for token_id in ids]
    return sp.decode(safe_ids)

# Use the safe decoder after generating outputs
output_ids = outputs[0].tolist()
decoded_text = safe_decode_with_sentencepiece(output_ids, sp)
print("Decoded Text:", decoded_text)


Decoded Text: Boston Va Phone number ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ 
